In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import lfilter

In [ ]:
## SRS (ISO 18431-4)
def srs_iso(ft,th,dr,dt):
    nf = len(ft);
    rs = np.zeros(nf);
    for i in range(nf):
        rs[i]= max(abs(SDOF_ISO(th,dt,ft[i],dr)));
    return ft,rs;

In [ ]:
# SDOF Transient Response (ISO 18431-4)
# th_in : acceleration time history at base
# th_out : acceleration time history at mass
def SDOF_ISO(th_in,dt,fn,zeta):

    wn = 2*np.pi*fn;
    wd = wn*np.sqrt(1-zeta**2);
    A=wn*dt*zeta;
    B=wd*dt;
    
    a, b = np.zeros(3), np.zeros(3);

    b[0] = 1 - np.exp(-A)*np.sin(B)/B;
    b[1] = 2*np.exp(-A)*(np.sin(B)/B - np.cos(B));
    b[2] = np.exp(-2*A) - np.exp(-A) * np.sin(B)/B;

    a[0] = 1;
    a[1] = -2*np.exp(-A)*np.cos(B);
    a[2] = np.exp(-2*A);

    th_out=lfilter(b, a, th_in, axis=-1, zi=None);
    
    return th_out;

In [ ]:
def fileO(x,y,filename):
    f = open(filename,'w');
    #f = open(filename,'a'); # overwrite
    for i in range(len(y)):
        f.write('%.4e\t%.4e\n'%(x[i],y[i]));
    f.close();

In [ ]:
# SDOF Transient Response (Smallwood)
# th_in : acceleration time history at base
# th_out : acceleration time history at mass
# out_opt : 1 = absolute acceleration / 2 = relative displacement
def SDOF_smallwood(th_in,dt,fn,zeta,out_opt):
    
    wn = 2*np.pi*fn;
    wd = wn*np.sqrt(1-zeta**2);
    a, b = np.zeros(3), np.zeros(3);
    E=np.exp(-zeta*wn*dt);
    E2=np.exp(-2*zeta*wn*dt);
    K=wd*dt;
    C=E*np.cos(K);
    P=2*zeta**2-1
    S=E*np.sin(K);
    Sp = S/K;
    if out_opt ==1:

        b[0] = 1 - Sp;
        b[1] = 2*(Sp-C);
        b[2] = E**2 - Sp;

        a[0] = 1;
        a[1] = -2*C;
        a[2] = E**2;

        th_out=lfilter(b, a, th_in, axis=-1, zi=None);
    else:
        a[0]=1         
        a[1]=-2*C
        a[2]=+E**2    
        
        b00=2*zeta*(C-1)
        b01=S*wn/wd*P
        b02=wn*dt
            
        b10=-2*wn*dt*C
        b11= 2*zeta*(1-E2)
        b12=-2*b01   
    
        b20=(2*zeta+wn*dt)*E2
        b21= b01
        b22=-2*zeta*C               
            
        b[0]=b00+b01+b02
        b[1]=b10+b11+b12
        b[2]=b20+b21+b22
            
        b=-b/(wn**3*dt)
        
        th_out=lfilter(b, a, th_in, axis=-1, zi=None);
        th_out *=386.4;
    return th_out;        

In [ ]:
## SRS (Smallwood Method)
def srs_sw(ft,th,dr,dt):
    nf = len(ft);
    rs = np.zeros(nf);
    for i in range(nf):
        rs[i]= max(abs(SDOF_smallwood(th,dt,ft[i],dr,1)));
    return ft,rs;

In [ ]:
## SRS (Smallwood Method) - Pseudo Acceleration Output
def srs_sw_pa(ft,th,dr,dt):
    nf = len(ft);
    rs = np.zeros(nf);
    for i in range(nf):
        rs[i]= ((2*np.pi*ft[i])**2)*max(abs((SDOF_smallwood(th,dt,ft[i],dr,2))))/386.4;  
    return ft,rs;

In [ ]:
def fileI(filename):
    tmp = []
    f = open(filename,'r');
    lines = f.readlines()
    for line in lines:
        tmp.append(line);
    out=np.zeros([len(tmp),5]);
    for i in range(len(out)):
        out[i,:] = np.array(tmp[i].split()).astype(np.float64)
    return out;

In [ ]:
def w(m,n,x):
    x[x==0]=1e-20;
    return (np.exp(1j*n*2*np.pi*x)-np.exp(1j*m*2*np.pi*x))/(1j*(n-m)*2*np.pi*x);

In [ ]:
def wb(m,n,x):
    return np.conjugate(w(m,n,x));

In [ ]:
def a(m,n,k,time,th):
    ff = th*wb(m,n,time-k/(n-m));
    int_ff = np.trapz(ff,time);
    int_ff *= (n-m)
    return int_ff;

In [ ]:
def ab(m,n,k,time,th):
    ff = th*w(m,n,time-k/(n-m));
    int_ff = np.trapz(ff,time);
    int_ff *= (n-m)
    return int_ff;

In [ ]:
def cwt_newland(p,k,time,th):  # p = No. of bands / k = time shift (integer)
    def w(m,n,x):
        x[x==0]=1e-20;
        return (np.exp(1j*n*2*np.pi*x)-np.exp(1j*m*2*np.pi*x))/(1j*(n-m)*2*np.pi*x);
    def wb(m,n,x):
        return np.conjugate(w(m,n,x));
    def a(m,n,k,time,th):
        ff = th*wb(m,n,time-k/(n-m));
        int_ff = np.trapz(ff,time);
        int_ff *= (n-m)
        return int_ff;
    def ab(m,n,k,time,th):
        ff = th*w(m,n,time-k/(n-m));
        int_ff = np.trapz(ff,time);
        int_ff *= (n-m)
        return int_ff;
    dt = time[1]-time[0]; fmax = 1/(2*dt);
    k1 = -k; k2 = k;
    bw = fmax/p;
    m = np.arange(0,fmax,bw)
    n = m+bw;
    a_out = np.zeros([len(m),2*k],'complex');
    ab_out = np.zeros([len(m),2*k],'complex');
    w_out = np.zeros([len(m),len(time),2*k],'complex');
    wb_out = np.zeros([len(m),len(time),2*k],'complex');
    for k in range(k1,k2):
        for i in range(len(m)):
            a_out[i,k] = a(m[i],n[i],k,time,th);
            ab_out[i,k] = ab(m[i],n[i],k,time,th);
            w_out[i,:,k] = w(m[i],n[i],time-k/(n[i]-m[i]));
            wb_out[i,:,k] = wb(m[i],n[i],time-k/(n[i]-m[i]));
    return a_out,ab_out,w_out,wb_out;

In [ ]:
def cwt_newland1(ft,k,time,th):  # ft = frequency table / k = time shift (integer)
    def w(m,n,x):
        x[x==0]=1e-20;
        return (np.exp(1j*n*2*np.pi*x)-np.exp(1j*m*2*np.pi*x))/(1j*(n-m)*2*np.pi*x);
    def wb(m,n,x):
        return np.conjugate(w(m,n,x));
    def a(m,n,k,time,th):
        ff = th*wb(m,n,time-k/(n-m));
        int_ff = np.trapz(ff,time);
        int_ff *= (n-m)
        return int_ff;
    def ab(m,n,k,time,th):
        ff = th*w(m,n,time-k/(n-m));
        int_ff = np.trapz(ff,time);
        int_ff *= (n-m)
        return int_ff;
    dt = time[1]-time[0]; fmax = 1/(2*dt);
    k1 = -k; k2 = k;
    m=ft[:-1]
    n=ft[1:]
    a_out = np.zeros([len(m),2*k],'complex');
    ab_out = np.zeros([len(m),2*k],'complex');
    w_out = np.zeros([len(m),len(time),2*k],'complex');
    wb_out = np.zeros([len(m),len(time),2*k],'complex');
    for k in range(k1,k2):
        for i in range(len(m)):
            a_out[i,k] = a(m[i],n[i],k,time,th);
            ab_out[i,k] = ab(m[i],n[i],k,time,th);
            w_out[i,:,k] = w(m[i],n[i],time-k/(n[i]-m[i]));
            wb_out[i,:,k] = wb(m[i],n[i],time-k/(n[i]-m[i]));
    return a_out,ab_out,w_out,wb_out;

In [ ]:
def icwt_newland(a,ab,w,wb):
    lenm = len(a[:,0]);
    lenk = len(a[0,:]);
    lent = len(w[0,:,0]);
    th = np.zeros(lent,'complex');
    for i in range(lenm):
        for j in range(lenk):
            th += a[i,j]*w[i,:,j]+ab[i,j]*wb[i,:,j];
    return th;